In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import scipy.ndimage as ndimage
import h5py

In [ ]:
import os
import sys

sys.path.append('/global/u1/l/lgupta/cori/GitHub/lcls-tools/')
sys.path.append('/global/u1/l/lgupta/cori/GitHub/lcls-tools/lcls_tools/devices/profile_monitor')
sys.path.append('/global/u1/l/lgupta/cori/GitHub/lcls-tools/lcls_tools/devices/magnet')
sys.path.append('/global/u1/l/lgupta/cori/GitHub/lcls-tools/lcls_tools/image_processing')

from lcls_tools import image_processing as imp

In [ ]:
import ImageProcessing as mip

In [ ]:
DATA_PATH = '/global/u1/l/lgupta/m669/datasets/Astra_EIC/MeasuredDataAstraSim/'
OCT_DATA_PATH = os.path.join(DATA_PATH + "oct_2019_vcc_database.h5")
NOV_DATA_PATH = os.path.join(DATA_PATH + "nov_2019_vcc_database.h5")

In [ ]:
f = h5py.File(NOV_DATA_PATH, "r")
vcc_timestamps = list(f['lcls_sc_image_and_pv_data_vcc'].keys())
print("Number of unique VCC images: " +str(len(vcc_timestamps)))
yag_timestamps = list(f['lcls_sc_image_and_pv_data_yag'].keys())
print("Number of unique VCC images: " +str(len(yag_timestamps)))

test = dict((f['lcls_sc_image_and_pv_data_vcc'][vcc_timestamps[0]]["image"].attrs))
VCC_resolution = test['res']
print("VCC image resolution in micron per pixel: " +str(VCC_resolution))

test = dict((f['lcls_sc_image_and_pv_data_yag'][yag_timestamps[0]]["image"].attrs))
YAG_resolution = test['res']
print("YAG image resolution in micron per pixel: " +str(YAG_resolution))


In [ ]:
YAGPVsofInterest=[
 'ACCL:GUNB:455:AACT',
 'LGPS:GUNB:1:BACT',
 'IRIS:LGUN:600:MOTR_ANGLE',
 'SOLN:GUNB:100:BACT',
 'SOLN:GUNB:212:BACT',
 'SOLN:GUNB:823:BACT',
 'TORO:GUNB:360:CHRG']

PVsofInterest=[
 'ACCL:GUNB:455:AACT',
 'LGPS:GUNB:1:BACT',
 'IRIS:LGUN:600:MOTR_ANGLE',
 'SOLN:GUNB:100:BACT',
 'SOLN:GUNB:212:BACT',
 'SOLN:GUNB:823:BACT']


In [ ]:
VCC_DATASET = {}
YAG_DATASET = {}
VCCpvs =[]
YAGpvs =[]
## VCC
for time in vcc_timestamps:
    VCCpvs.append(mip.getPVsVCC(PVsofInterest, time, f))
    mip.makeDataPerVCC(PVsofInterest, VCC_DATASET, time, f)
    
## YAG
for time in yag_timestamps:
    YAGpvs.append(mip.getPVsYAG(PVsofInterest, time, f))
    mip.makeDataPerYAG(PVsofInterest, YAG_DATASET, time, f)
VCCpvs = np.array(VCCpvs)
YAGpvs = np.array(YAGpvs)

In [ ]:
timestamp_pairs = {}
viable_vccs = []
matches = 0
for vt in vcc_timestamps[:]:
    
    vcc = np.array(VCC_DATASET[vt])
    stamps = []
    for yt in yag_timestamps[:]:
        #print(yt)
        if yt!="2019-11-01T13:46:54.311939" and yt!="2019-11-01T13:48:51.510801" and yt!="2019-11-01T13:38:44.399731":
            yag = np.array(YAG_DATASET[yt])
            percent_dif = 100*np.abs((vcc-yag)/yag)
            if np.all(percent_dif[1:] < 10):
                stamps.append(yt)

            timestamp_pairs[vt] = stamps

    
viable_VCCs = []

for t in timestamp_pairs.keys():
    x = len(timestamp_pairs[t])
    if x > 0:
        matches = matches+x
        viable_VCCs.append(t)
print(matches)

In [ ]:
actually_viable_VCCs = []


for vt in viable_VCCs:
    if vt != "2019-11-01T12:19:15.316264":
        VCCimage = np.array(f['lcls_sc_image_and_pv_data_vcc'][vt]["image"][:])
        img, masked, xproj, xdata, xgauss_fit, xcov, xerr, yproj, ydata, ygauss_fit, ycov, yerr = mip.VCCAnalysis(VCCimage, plotting = False, resolution = VCC_resolution)
        if np.sum(masked) > 7000.0:
            #print(np.sum(masked))
            actually_viable_VCCs.append(vt)
            #print(vt)
            #plt.imshow(img)
            #plt.show()
        
print(np.shape(actually_viable_VCCs))

In [ ]:
yagPVs = []
vccPVs = []

plotting = False

first = 0
i = 0
for vt in actually_viable_VCCs:

    yts = timestamp_pairs[vt]
    for yt in yts:
        
        if first == 0:
            VCCimages = np.array(f['lcls_sc_image_and_pv_data_vcc'][vt]["image"][:])
            YAGimages = np.array(f['lcls_sc_image_and_pv_data_yag'][yt]["image"][:, :])
            first = 1
        else:
            vccimg = np.array(f['lcls_sc_image_and_pv_data_vcc'][vt]["image"][:])
            VCCimages = np.dstack((VCCimages, vccimg))
            yagimage = np.array(f['lcls_sc_image_and_pv_data_yag'][yt]["image"][:])
            YAGimages = np.dstack((YAGimages, yagimage[:, :]))
            #YAGimages = np.dstack((YAGimages, yagimage[50:, 900]))
            if plotting:
                plt.figure(figsize=(2,2))
                print(i, yt)
                plt.imshow(YAGimages[:,:,-1])
                plt.show()
        i = i + 1
        first = 1
        YAGpvs = mip.getPVsYAG(YAGPVsofInterest, yt, f)
        vccPVs.append(mip.getPVsVCC(YAGPVsofInterest,vt,f))
        yagPVs.append(YAGpvs)
        
yagPVs = np.array(yagPVs)
vccPVs = np.array(vccPVs)
print(np.shape(VCCimages))        
print(np.shape(YAGimages)) 
print(np.shape(yagPVs))
print(np.shape(vccPVs))

In [ ]:
np.shape(vccPVs)

In [ ]:
np.shape(yagPVs)

In [ ]:
VCC_names = []
YAG_names = []
for vt in actually_viable_VCCs:
    yts = timestamp_pairs[vt]
    print( vt, len(yts))
    for i in range(len(yts)):
        VCC_names.append(vt)
        YAG_names.append(yts[i])
        
        

In [ ]:
VCC_analysis = mip.doAnalysisAndReturnDict(VCCimages, plotting = False, resolution = VCC_resolution, VCC = True)    
YAG_analysis = mip.doAnalysisAndReturnDict(YAGimages, plotting = True, npix = 150, resolution = YAG_resolution, VCC = False, verbose = False)  

In [ ]:
#import ImageProcessing as mip
inps, vbin, ybin, yfit = mip.package(VCC_analysis, vccPVs, VCC_resolution, YAG_analysis, yagPVs, YAG_resolution, YAGPVsofInterest)

In [ ]:
## make text files and yaml files


In [ ]:


savepath = "/global/u1/l/lgupta/m669/datasets/Astra_EIC/MeasuredDataAstraSim/VCC/distgen/"
savenames = []

for ind, name in enumerate(VCC_names):
    name = name.split('.')[0]
    savename = savepath+name
    image = VCC_analysis['masked_images'][ind]
    n,m = np.shape(image)
    np.savetxt(savename+'.txt',image)
    xwidth=VCC_resolution*n*(1E-3)
    ywidth=VCC_resolution*m*(1E-3)
    xcm=VCC_resolution*(n/2)*(1E-3)
    ycm=VCC_resolution*(m/2)*(1E-3)
    
    with open(savename+'.txt', 'r+') as f:
        with open(savename+"_out.txt", "w") as f2:
            line = 'x ' + str(xwidth) + " " + str(xcm) + " [mm]\ny " + str(ywidth) + " " + str(ycm) + " [mm]\n"
            f2.write(line)
            for line in f:
                f2.write(line)
    
    savenames.append(savename+"_out.txt")
    
    

In [ ]:
import yaml
####for making yaml input files DO NOT DELETE
originalfile = "/global/homes/l/lgupta/m669/MeasuredData/VCC/laser_in.yaml"
with open(originalfile) as j:
    originaldata = yaml.load(j, Loader=yaml.FullLoader)
    
yaml_input_files = []
for ind, name in enumerate(savenames):
    data = originaldata
    data["xy_dist"]["file"] = name
    data["start"]["MTE"]["value"] = 250
    outname = name.split('_out')[0] + "_laser_in.yaml"
    yaml_input_files.append(outname)
    with open(outname, 'w') as f:
        yaml.dump(data, f)

print("Files written to:  "+savepath)

In [ ]:
inps["distgen_inputs"] = yaml_input_files

In [ ]:
inps.keys()

In [ ]:
import json
from json import JSONEncoder


class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
    
with open("/global/u1/l/lgupta/m669/datasets/Astra_EIC/MeasuredDataAstraSim/nov_inputs.json", 'w') as f:
    json.dump(inps,f, cls=NumpyArrayEncoder)

np.save("/global/u1/l/lgupta/m669/datasets/Astra_EIC/MeasuredDataAstraSim/nov_vcc_binned.npy", vbin)
np.save("/global/u1/l/lgupta/m669/datasets/Astra_EIC/MeasuredDataAstraSim/nov_yag_binned.npy", ybin)
np.save("/global/u1/l/lgupta/m669/datasets/Astra_EIC/MeasuredDataAstraSim/nov_yag_fits.npy", yfit)

In [ ]:
for i in YAGPVsofInterest:
    plt.hist(inps[i])
    plt.title(i)
    plt.show()